# Scheduling Engine 入門チュートリアル

<a id="concept"></a>
## Scheduling Engine (SE) とは

**Amplify Scheduling Engine (SE)** は、多岐にわたるスケジュールの立案・最適化に特化した最適化エンジンです。どなたでも定式化不要で、最適化がなされた種々の計画立案が可能です。柔軟に設計された専用ライブラリにより、多品種少量や変種変量生産のような複雑で迅速、頻繁な立案が求められる生産計画立案に最適です。また、生産計画以外にも『シフト計画（今後サンプル追加予定）』や『[配送計画](https://amplify.fixstars.com/ja/scheduling/resources/example/vrp)』といった種々の計画問題もカバーします。

以下では、Fixstars Amplify SE 及びそれを使った最適化実施をサポートするライブラリ `amplify-sched` について解説します。本ライブラリは、Python ライブラリであり、

```shell
pip install amplify-sched
```

でどなたでも[利用をスタート](https://amplify.fixstars.com/ja/scheduling/resources/quickstart)できます。

<a id="scheduling"></a>
## スケジューリング問題

スケジューリング問題とは、複数の『ジョブ (Job) 』とそれらを処理する複数の『マシン (Machine) 』利用のスケジュールを決定する問題です。Fixstars Amplify SE は幅広いスケジューリング問題に適用可能ですが、ここでは具体例として、クッキーを製造する工場における生産計画問題を考えましょう。この工場では、

- プレーンクッキ
- チョコチップクッキー
- ジンジャークッキー

を製造しています。つまり、この工場では、3種類のクッキーを製造するという3つのジョブを実行する必要があります。通常、それぞれのジョブは複数の『工程』 (Process) から構成されており、ここでは、

0. 生地を作る
1. クッキーの型を取る
2. 焼く
3. 梱包する
 
という工程があるとします（後のコードとの整合性の為、0 番からスタートしています）。

生産過程において、最も細分化された処理単位として、タスク (Task) があります。同じ工程に属するタスクであっても、例えば、プレーンクッキー用の生地作りというタスクと、ジンジャークッキー用の生地作りというタスクは別のタスクです。ジョブと工程の組合せの数だけ処理するタスクが存在します。

これらのタスクは、様々なマシンによって処理されますが、マシンの性能及び汎用性により、処理方法が異なります。例えば、あるマシンは全ジョブのある工程に属するタスクを処理（例：全クッキーの型取りを実施）できるかもしれません。また、別のマシンは複数種類のタスクを処理（例：プレーンクッキー用の生地作りとジンジャークッキー用の型取りを実施）できるかもしれません。

このようなマシン情報とジョブ情報が与えられたとき、どのタスクをどのマシンでどのタイミングで処理し、全てのジョブを完了するか、という計画立案課題に対し、実行可能な計画の立案だけに留まらず、メイクスパンを最小化するような最適化まで行うのが Fixstars Amplify SE となります。

次に、Fixstars Amplify SE の利用に必要な

- [ジョブ (Job)](#job)
- [タスク (Task)](#task)
- [マシン (Machine)](#machine)
- [処理時間 (Processing Time)](#proc_time)
- [メイクスパン (Makespan)](#makespan)

の概念についてもう少し掘り下げて説明します。その後、複数の計画立案例についてサンプルコードとともに[解説](#tutorial)します。

これらの用語は業界・領域により異なる呼び方をされる場合もありますが、適宜読み替えて下さい。読み替えが必要な典型例として、『[マシニングセンターのスケジュール問題](https://amplify.fixstars.com/ja/scheduling/resources/example/machining_center)』を紹介しています（発展的な問題ですので、本 SE 入門チュートリアル修了後に取り組んでください）。

<a id="job"></a>
- ### ジョブ (Job) とは

  上記クッキー工場の例では、3 種類のクッキーを製造するという 3 つのジョブが存在します。そして、各ジョブは複数のタスクから構成されます。

  各ジョブ内のタスクの順番は工程 (Process) を使って表現されます（例：工程 0、工程 1、工程 2、・・・）。上記クッキー工場の例では、工程 0 には、各クッキーの生地を作るタスク、また工程 1 には、それぞれのクッキーの型を取るタスクが該当し、それぞれのクッキー（ジョブ）に対し、この順番でタスクが処理されます。

  ![job_table](./figures/job_table.drawio.svg)

<a id="task"></a>
- ### タスク (Task) とは

  タスクは、ジョブと工程から一意に決まります。例えば、クッキー工場の例では、プレーンクッキーを製造するジョブ 0 の工程 1 のタスクは、プレーンクッキー生地の型取りのタスク、であることが分かります。各タスクが持つ情報として、処理開始時刻、処理終了時刻、処理を実施するマシン名があります。

  ![Task](./figures/task.drawio.svg)

<a id="machine"></a>
- ### マシン (Machine) とは

  マシンは、ジョブを構成するタスクを処理する機械です。1つのマシンで複数のタスクを同時に処理することはできません。また、ジョブ `j` 工程 `p` に対応するタスク `(j, p)` の処理を行うことができるマシンごとに、当該タスクの処理に必要な時間（処理時間）が決まっています。

  ![Machine](./figures/machine.drawio.svg)

<a id="proc_time"></a>
- ### 処理時間 (Processing Time) とは

  処理時間とは、あるタスクの処理を行うことができるマシンが、そのタスクを完了するのにかかる時間です。

<a id="makespan"></a>
- ### メイクスパン (Makespan) とは

  メイクスパンとは、開始時刻からすべてのジョブの最後の処理終了時刻までの時間です。Fixstars Amplify SE で立案される種々のスケジュールは、メイクスパンができるだけ最小になる様に最適化されます。

  ![makespan](./figures/makespan.drawio.svg)

<a id="tutorial"></a>
## 最適化スケジューリング・チュートリアル

ここまでの事前知識のみで、最適化が施された生産計画の立案が可能です。上記のクッキー工場を例にとり、よくある 3 つの運用ケースを想定した以下チュートリアルを順にご覧ください。ここでは、`amplify-sched` を使ったサンプルプログラムも公開・解説しています。

- **最もシンプル**な生産計画立案  
  各マシンが単一の工程を処理する場合のスケジューリング問題 (Flow Shop Scheduling)  $\rightarrow$ [チュートリアル](https://amplify.fixstars.com/ja/scheduling/resources/tutorial/se_tutorial_flow_shop)

- **少し柔軟**な生産計画立案  
  各タスクに対して、担当できる1台のマシンが決まっている場合のスケジューリング問題 (Job Shop Scheduling)  $\rightarrow$ [チュートリアル](https://amplify.fixstars.com/ja/scheduling/resources/tutorial/se_tutorial_job_shop)

- **より柔軟**な生産計画立案  
  各タスクを処理可能なマシンが複数ある場合のスケジューリング問題 (Flexible Job Shop Scheduling)  $\rightarrow$ [チュートリアル](https://amplify.fixstars.com/ja/scheduling/resources/tutorial/se_tutorial_flexible_job_shop)


さらに複雑な条件を考慮する場合に必要な制約についての説明は、引き続き次節『[制約](#constraint)』へお進みください。また、これらの制約を使いこなした、最適化された生産計画立案を実現するサンプルプログラム例は[サンプルコード](https://amplify.fixstars.com/ja/scheduling/resources#code)をご覧ください。

<a id="constraint"></a>
## 制約

スケジュール立案に際して、メイクスパンが最小となる様に探索が実施されますが、さらに次に述べる様々な制約を付加的に考慮することも可能です。制約条件は複数組み合わせて利用することができます。

<a id="release_time_deadline"></a>
- ### 開始可能時刻、終了締め切り時刻

  各タスクには開始可能時刻 (`release_time`) と終了締め切り時刻 (`deadline`) を与えることができます。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#release-time-deadline)]

  ![release_time](./figures/release_time.drawio.svg)

  ![deadline](./figures/deadline.drawio.svg)

<a id="transportation_time"></a>
- ### 輸送時間

  あるマシンから別のマシンへの部品などの輸送時間 (`transportation_time`) をタスクごとに設定できます。輸送時間が設定されている場合、その時間より短い間隔では次のタスクを開始できません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#transportation-time)]

  ![transportation_time](./figures/transportation_time.drawio.svg)

<a id="setup_time"></a>
- ### 準備時間

あるマシンにおいて、あるタスクから別のタスクへの切り替えにかかる準備時間 (`setup_time`) を設定できます。準備時間が設定されている場合、その時間より短い間隔では次のタスクを開始できません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#setup-time)]

![setup_time](./figures/setup_time.drawio.svg)

<a id="maintenance_time"></a>
- ### メンテナンス時間

あるマシンが稼働できない時間区間 (`maintenance_time`) を設定できます。設定した時間区間ではそのマシンではタスクを処理することはできません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#maintenance-time)]

![maintenance_time](./figures/maintenance_time.drawio.svg)

<a id="buffer"></a>
- ### バッファ

あるマシンで終了したタスクを、次のマシンでの処理が始まるまで保管する倉庫を考えます。このとき、各マシンの倉庫の容量 (`buffer_size`) と各タスクが占有するサイズ (`required_buffer_size`) を設定することができます。これらが設定されている場合、各時刻において倉庫の容量を超えるようなスケジュールは許されません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#buffer)]

![buffer](./figures/buffer.drawio.svg)

<a id="resource"></a>
- ### リソース

各タスクを行うために必要なリソース (`required_resource`) を定義できます。各リソースには同時使用上限数 (`capacity`) が定められ、その数を超える複数のタスクで同時にリソースを使用できません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#resource)]

![resource](./figures/resource.drawio.svg)

<a id="no_wait"></a>
- ### 待ち無し

各ジョブに no-wait フラグ (`no_wait`) を設定できます。no-wait が設定されたジョブは一度タスクを始めたら、全タスクが終了するまで連続でタスクを行う必要があります。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#no-wait)]

![no_wait](./figures/no_wait.drawio.svg)

<a id="dependent_jobs"></a>
- ### ジョブの依存関係

各ジョブごとに依存するジョブ (`dependent_jobs`) を複数定義できます。依存するジョブが定義されている場合、それらの依存ジョブがすべて完了しないと、そのジョブは開始できません。[[関連ドキュメントページ](https://amplify.fixstars.com/ja/docs/amplify-se/notebook/constraints.html#dependent-jobs)]

![dependent_jobs](./figures/dependent_jobs.drawio.svg)